In [25]:
import pandas as pd
import numpy as np
from khaiii import KhaiiiApi

In [26]:
data = pd.read_csv("./modi_data/kor_full.csv")
data.head()

,no,year,title,kor_full
0,1,1997,멀티미디어 원격교육에 관한 연구,멀티미디어 원격교육에 관한 연구
1,2,1997,교육용 하이퍼미디어 자료 편집기에 관한 연구,교육용 하이퍼미디어 자료 편집기에 관한 연구
2,3,1997,인터넷 기반의 코스웨어의 설계 및 구현,인터넷 기반의 코스웨어의 설계 및 구현
3,4,1997,Web에서의 협력 환경 구축 방안 연구,Web에서의 협력 환경 구축 방안 연구
4,5,1997,열린교육에서의 개별화수업과 CAI,열린교육에서의 개별화수업과 CAI


In [27]:
dfWordList = pd.read_excel("./khaiii_word_cor.xlsx")
dfWordList2 = pd.read_excel("./khaiii_word_cor_etc.xlsx")
#print(dfWordList2)

dfWordDel = dfWordList[dfWordList["수정"] == "삭제"]
dfWordMod = dfWordList[dfWordList["수정"] != "삭제"]
dfWordDiv = dfWordList2
#print(dfWordMod)

seriesDelete = dfWordDel["기존"]
stopword = []
for word in seriesDelete.values:
    stopword.append(word)
#print(stopword)

seriesModify = dfWordMod["기존"]
modiword = []
for word in seriesModify.values:
    modiword.append(word)
#print(len(modiword))

seriesModify2 = dfWordMod["수정"]
modiword2 = []
for word in seriesModify2.values:
    modiword2.append(word)
#print(len(modiword2))

seriesDivide = dfWordDiv["기존"]
divword = []
for word in seriesDivide.values:
    divword.append(word)
#print(divword)
#print(len(divword))

seriesDivide2 = dfWordDiv["수정"]
divword2 = []
for words in seriesDivide2.values:
    divword2.append(words.split(', '))
#print(divword2)
#print(len(divword2))

In [28]:
api = KhaiiiApi()
def khaiiiTokenizer(raw, stopword=stopword, pos=['NNG', 'NNP', 'NNB', 'NP', 'NR', 'SL']): # 일반명사 고유명사 의존명사 대명사 수사 외국어
    list = []
    skip = 0

    for word in api.analyze(raw): #raw data
        #print(word)
        
        for i, morph in enumerate(word.morphs):
            #print(morph.lex)
            if skip == 1: 
                #print(morph.lex) # '지능'
                skip = 0
                continue

            if morph.lex == '인공' and i+1 < len(word.morphs) and word.morphs[i+1].lex == "지능":
                #print(morph.lex + word.morphs[i+1].lex) # 인공지능
                list.append(morph.lex + word.morphs[i+1].lex)
                skip = 1
                continue

            if len(morph.lex) > 1 and morph.tag in pos and morph.lex not in stopword: 
                if morph.tag == 'SL':
                    morph.lex = morph.lex.lower()
                if morph.lex in divword:
                    morph.lex = divword2[divword.index(morph.lex)]
                    list.extend(morph.lex)
                elif morph.lex in modiword:
                    morph.lex = modiword2[modiword.index(morph.lex)]
                    list.append(morph.lex)
                else: list.append(morph.lex)
                
    return list

In [29]:
tokenized = data["kor_full"].apply(lambda row: khaiiiTokenizer(row))
print(tokenized)
#tokenized.to_csv("./modi_data/token_khaiii.csv")
print("========= tokenization completed =========")

0                                     [멀티미디어, 원격, 교육, 연구]
1                               [교육, 하이퍼미디어, 자료, 편집기, 연구]
2                                     [인터넷, 코스웨어, 설계, 구현]
3                               [web, 협력, 환경, 구축, 방안, 연구]
4                                       [교육, 개별, 수업, cai]
                              ...                        
1144    [개정, 교과서, 소프트웨어, 교육, 단원, 탐구, 비교, 분석, 교육, 과정, 교...
1145    [이러닝, 콘텐츠, 사용자, 경험, ux, 평가, 이러닝, 대리, 상호, 작용, 사...
1146    [초등, 데이터, 리터러시, 함양, ai, 데이터, 과학, 교육, 프로그램, 개발,...
1147    [초등, 예비, 교사, 소프트웨어, 교육, 온라인, 교육, 효과, 분석, 소프트웨어...
1148    [초등, 교과서, 소프트웨어교육, 영역, 컴퓨팅, 사고력, 요소, 분석, 소프트웨어...
Name: kor_full, Length: 1149, dtype: object
========= tokenization completed =========


In [30]:
# n-gram candidates

def get_ngrams(raw, n_range=(2,3)): # 2~3-gram 까지

    def to_ngrams(words, n):
        ngrams = []
        for b in range(0, len(words) - n + 1):
            ngrams.append(tuple(words[b:b+n]))
        return ngrams

    n_begin, n_end = n_range
    ngram_list = []
    
    for n in range(n_begin, n_end + 1):
        for ngram in to_ngrams(raw, n):
            ngram_list.append(ngram)
            
    return ngram_list


In [31]:
ngrams = tokenized.apply(lambda row: get_ngrams(row))

print(ngrams)

#ngrams.to_csv('./modi_data/ngrams.csv')

0       [(멀티미디어, 원격), (원격, 교육), (교육, 연구), (멀티미디어, 원격, ...
1       [(교육, 하이퍼미디어), (하이퍼미디어, 자료), (자료, 편집기), (편집기, ...
2       [(인터넷, 코스웨어), (코스웨어, 설계), (설계, 구현), (인터넷, 코스웨어...
3       [(web, 협력), (협력, 환경), (환경, 구축), (구축, 방안), (방안,...
4       [(교육, 개별), (개별, 수업), (수업, cai), (교육, 개별, 수업), ...
                              ...                        
1144    [(개정, 교과서), (교과서, 소프트웨어), (소프트웨어, 교육), (교육, 단원...
1145    [(이러닝, 콘텐츠), (콘텐츠, 사용자), (사용자, 경험), (경험, ux), ...
1146    [(초등, 데이터), (데이터, 리터러시), (리터러시, 함양), (함양, ai),...
1147    [(초등, 예비), (예비, 교사), (교사, 소프트웨어), (소프트웨어, 교육),...
1148    [(초등, 교과서), (교과서, 소프트웨어교육), (소프트웨어교육, 영역), (영역...
Name: kor_full, Length: 1149, dtype: object


In [39]:
ngram_list = []

for i in range(len(ngrams)):
    #print(ngrams[i])
    ngram_list.extend(ngrams[i])
print(ngram_list)

, ('적용', '프로그램'), ('프로그램', '효과'), ('효과', '분석'), ('분석', '분석'), ('분석', '결과'), ('결과', '초등'), ('초등', '예비'), ('예비', '교사'), ('교사', '대상'), ('대상', '데이터'), ('데이터', '시각'), ('시각', '교육'), ('교육', '중심'), ('중심', '스크래치'), ('스크래치', '활용'), ('활용', '소프트웨어'), ('소프트웨어', '교육'), ('교육', '초등'), ('초등', '예비'), ('예비', '교사'), ('교사', '창의'), ('창의', '하위'), ('하위', '요소'), ('요소', '유창'), ('유창', '정교'), ('정교', '창의'), ('창의', '평균'), ('평균', '창의'), ('창의', '지수'), ('지수', '유의미'), ('유의미', '효과'), ('효과', '확인'), ('스크래치', '활용', '데이터'), ('활용', '데이터', '시각'), ('데이터', '시각', '교육'), ('시각', '교육', '초등'), ('교육', '초등', '예비'), ('초등', '예비', '교사'), ('예비', '교사', '창의'), ('교사', '창의', '영향'), ('창의', '영향', '데이터'), ('영향', '데이터', '시각'), ('데이터', '시각', '창의'), ('시각', '창의', '스크래치'), ('창의', '스크래치', '예비'), ('스크래치', '예비', '교사'), ('예비', '교사', '연구'), ('교사', '연구', '데이터'), ('연구', '데이터', '시각'), ('데이터', '시각', '교육'), ('시각', '교육', '초점'), ('교육', '초점', '스크래치'), ('초점', '스크래치', '초등'), ('스크래치', '초등', '예비'), ('초등', '예비', '교사'), ('예비', '교사', '적용'), ('교사', '적용', '창의'), ('적용', '창

In [ ]:
from collections imporrt defaultdict

def get_ngram_counter(docs, min_count=10):
    pass

In [ ]:
def get_ngram_score(ngram_list, delta=30):
    ngrams = []

    for ngram in ngram_list:
        first = ngram_list[ngram[:-1]]
        second = ngram_list[ngram[1:]]
        score = 